In [77]:
%matplotlib notebook
from IPython.display import display, clear_output
# Initializing Temperature Reader part

from mcculw import ul
from mcculw.enums import ULRange
from mcculw.ul import ULError
import time, csv
import numpy as np
# Initializing Origin

import sys
import os.path
import ConfigParser
import pprint
import logging
import time

#fullBinPath = os.path.abspath(os.getcwd() + "/" + sys.argv[0])
fullBinPath = "C:\\github\\Origin\\bin\\" # Path to Origin bin
fullBasePath = os.path.dirname(os.path.dirname(fullBinPath))
fullCfgPath = os.path.join(fullBasePath, "config")
fullLibPath  = os.path.join(fullBasePath, "lib")
sys.path.append(fullLibPath)

from origin.client import server
from origin import current_time, TIMESTAMP
configfile = os.path.join(fullCfgPath, "origin-server.cfg")

import ConfigParser
config = ConfigParser.ConfigParser()
config.read(configfile)

# something that represents the connection to the server
serv = server(config)

In [91]:
hexagon_path='Y:/Rb/Data/old temp data/'
proxy_csv_filename='Rb_8thermistors_proxy.csv'
csv_filename='Rb_8thermistors.csv'

filenames=[csv_filename,hexagon_path+csv_filename, hexagon_path+proxy_csv_filename]

print "registering stream..."
connection = serv.registerStream(
  stream="Rb_Thermistors",
  records={
      "T_East":"float", "R_East":"float",
      "T_South":"float", "R_South":"float",
      "T_West":"float", "R_West":"float",
      "T_North":"float", "R_North":"float",
      "T_Table":"float", "R_Table":"float",
      "T_Red_plate":"float", "R_Red_plate":"float",
      "T_Blue_plate":"float", "R_Blue_plate":"float",
      "T_Radiator":"float", "R_Radiator":"float",
})
print "success"

registering stream...
0       
successfully registered with streamID: 22, version: 1
success


In [71]:
def get_thermistor_reading():
    board_num=0 # Board number defined from Instacal software. Normally "0".
    low_chan, high_chan=0, 7
    ai_range = ULRange.BIP5VOLTS # To readout in Celcius
    NOSCALE=5 # To readout in resistance
    try:
        # Get values from the device
        T_values = ul.t_in_scan(board_num, low_chan, high_chan, ai_range)
        R_values = ul.t_in_scan(board_num, low_chan, high_chan, NOSCALE)
        # Display the value, if you wish to monitor.
        #print "T (C): {}".format(T_values.data_array[:])
        #print "R (Ohm): {}".format(R_values.data_array[:])
    except ULError as e:
        # Display the error
        print("A UL error occurred. Code: " + str(e.errorcode)
              + " Message: " + e.message)
    return T_values.data_array[:], R_values.data_array[:]

def organize_data():
    channels=range(0,8)
    labels=["East","Red_plate","Blue_plate","Table","North","West","South","Radiator","Blue_place"]
    Temperatures, Resistances =get_thermistor_reading()
    ts=time.time()
    
    data = { TIMESTAMP: ts} # initialize the data
    for channel in channels:
        data["T_"+labels[channel]]=Temperatures[channel]
        data["R_"+labels[channel]]=Resistances[channel]
    
    return data, Temperatures, Resistances, ts

def send_data():
    data, Temperatures, Resistances, ts = organize_data()
    display(data)
    connection.send(**data)

In [ ]:
# Now let's initiate sending the data to the server. We'll also make a local, remote and remote proxy.
while(True):
    data, Temperatures, Resistances, ts = organize_data()
    # CSV file organized by channel 0 - > 7
    l = [[ts],Temperatures, Resistances]
    flat_list = [item for sublist in l for item in sublist]
    
    display(data)
    try:
        display("Last sent at time : {}".format(time.time()))
        connection.send(**data)
    except:
        display("Failed at time : {}".format(time.time()))

    for csv_file in filenames:
        try:
            with open(csv_file,'ab') as myFile:
                FileWriter = csv.writer(myFile)
                FileWriter.writerow(flat_list)
        except:
            print "{} file might be used from other place. Please close it".format(csv_file)
    
    time.sleep(1)
    clear_output()